# Behavior Analysis: Duration Per Behavior Bout

Press SHIFT + ENTER to run code

### USER INPUT!

In [3]:
## Define project
project_name = ''

# Select group and condition, see meta.py to see how you previously defined it for your project
selected_groups = ['', '']
selected_conditions = ['', '']

### Main Code: Create Bar Graph of Duration Per Behavior Bout

In [4]:
###### RUN DEPENDENCIES ######
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import warnings
import seaborn as sns
import matplotlib

# Add parent directory to path if needed
if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))

from utils.classification import load_behaviors
from utils.meta import *

###### Load Behaviors ######
behaviors = load_behaviors(f"../processed_dataset/{project_name}/behaviors_{project_name}.pkl")

# Define the directory path for saving figures
directory_path = f"../processed_dataset/{project_name}/figures/behavior_instance-durations"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

def get_duration_bouts(predict, behavior_classes, framerate=60):
    behav_durations = []
    bout_start_idx = np.where(np.diff(np.hstack([-1, predict])) != 0)[0]
    bout_durations = np.hstack([np.diff(bout_start_idx), len(predict) - np.max(bout_start_idx)])
    bout_start_label = predict[bout_start_idx]
    for b, behavior_name in enumerate(behavior_classes):
        idx_b = np.where(bout_start_label == int(b))[0]
        if len(idx_b) > 0:
            behav_durations.append(bout_durations[idx_b] / framerate)
        else:
            a = np.empty((1,))
            a[:] = np.nan
            behav_durations.append(a)
    return behav_durations

# Set up the grid dimensions.
# total_rows: one row per selected group, total_cols: maximum number of columns
total_rows = len(selected_groups)
total_cols = len(selected_conditions)

fig, ax = plt.subplots(total_rows, total_cols, figsize=(10, 11), sharex=False, sharey=True)

# Reshape the axes array to 2D if needed so we can always index with [row, col]
if total_rows == 1 and total_cols == 1:
    ax = np.array([[ax]])
elif total_rows == 1:
    ax = np.expand_dims(ax, axis=0)
elif total_cols == 1:
    ax = ax.reshape(-1, 1)

# Loop through each row corresponding to each selected group
for row in range(total_rows):
    # Use a local variable to control number of columns to plot in this row.
    # For example, if row > 4 then only plot one column.
    n_plot_cols = total_cols if row <= 4 else 1

    # Remove extra axes from this row if n_plot_cols is less than the total number of columns
    if n_plot_cols < total_cols:
        for extra_col in range(n_plot_cols, total_cols):
            fig.delaxes(ax[row, extra_col])
    
    # Loop through each column to plot data for this row
    for col in range(n_plot_cols):
        # If only one column is displayed on the row, we always take the first condition.
        selected_group = selected_groups[row]
        selected_condition = selected_conditions[col] if n_plot_cols > 1 else selected_conditions[0]
        
        durations_ = []
        if selected_group in behaviors and selected_condition in behaviors[selected_group]:
            file_keys = list(behaviors[selected_group][selected_condition].keys())
            for file_name in file_keys:
                durations_.append(get_duration_bouts(behaviors[selected_group][selected_condition][file_name], behavior_names))
        
        # Prepare a DataFrame to plot
        durations_dict = {
            'behavior': np.hstack([np.hstack([np.repeat(behavior_names[i], len(durations_[f][i])) 
                                              for i in range(len(durations_[f]))]) 
                                   for f in range(len(durations_))]),
            'duration': np.hstack([np.hstack(durations_[f]) for f in range(len(durations_))]),
        }
        durations_df = pd.DataFrame(durations_dict)
        durations_df.to_csv(f"../processed_dataset/{project_name}/figures/behavior_instance-durations/behavior_durations_{selected_group}_{selected_condition}.csv", index=False)

        # Create a boxplot using seaborn
        sns.boxplot(
            data=durations_df, 
            x='duration', 
            y='behavior', 
            hue='behavior', 
            orient='h', 
            width=0.8,
            palette=behavior_colors, 
            showfliers=False, 
            ax=ax[row, col]
        )
        ax[row, col].set_ylabel('')
        ax[row, col].set_xlabel('')
        
        if row == total_rows - 1:
            ax[row, col].set_xlabel('Behavior duration (s)')
        ax[row, col].set_title(f'{selected_group} - {selected_condition}')
        
        # Set aspect and axis limits
        ax[row, col].set_aspect('equal', adjustable='box')
        ax[row, col].set_xlim(0, 6)
        
        # Remove the legend if present
        legend = ax[row, col].get_legend()
        if legend is not None:
            legend.remove()

# Adjust layout and save the figure
plt.tight_layout()
fig.savefig(f"../processed_dataset/{project_name}/figures/behavior_instance-durations/behavior_durations_{project_name}.svg", 
            dpi=600, bbox_inches='tight')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../processed_dataset/project_ACC_MiniscopeSNI_Female_3Month/behaviors_project_ACC_MiniscopeSNI_Female_3Month.pkl'

# ANALYSIS COMPLETE!

## Additional Analysis: Average Duration (seconds) of Behavioral Bouts Per File

In [ ]:
import numpy as np
import pandas as pd

# This list will hold the durations for all files
all_file_durations = []

for row in range(len(selected_groups)):
    for col in range(len(selected_conditions)):
        selected_group = selected_groups[row]
        selected_condition = selected_conditions[col]

        # Check if the selected group and condition are available
        if selected_group in behaviors and selected_condition in behaviors[selected_group]:
            file_keys = list(behaviors[selected_group][selected_condition].keys())
            
            for file_name in file_keys:
                # Get the duration bouts for the current file
                file_durations = get_duration_bouts(behaviors[selected_group][selected_condition][file_name], behavior_names)
                
                # Sum up all durations per behavior
                total_durations = [np.nansum(d) for d in file_durations]
                # Compute mean durations per behavior
                mean_durations = [np.nanmean(d) if not np.isnan(np.nanmean(d)) else 0 for d in file_durations]
                # Compute std durations per behavior
                std_durations = [np.nanstd(d) if not np.isnan(np.nanstd(d)) else 0 for d in file_durations]
                
                # Create a record with group, condition, filename, total/mean/std durations
                record = {
                    'group': selected_group,
                    'condition': selected_condition,
                    'file_name': file_name
                }
                
                # Add each behavior's total, average, and std duration to the record
                for i, behavior_name in enumerate(behavior_names):
                    record[f'{behavior_name}_total_duration_s'] = total_durations[i]
                    record[f'{behavior_name}_average_duration_s'] = mean_durations[i]
                    record[f'{behavior_name}_std_duration_s'] = std_durations[i]
                
                # Append the record to the list
                all_file_durations.append(record)

# Convert the list of records to a DataFrame
all_file_durations_df = pd.DataFrame(all_file_durations)

# Define the CSV file path and save the DataFrame
output_csv_path = f"../processed_dataset/{project_name}/figures/behavior_instance-durations/total_average_std_durations_per_file_{project_name}.csv"
all_file_durations_df.to_csv(output_csv_path, index=False)

print(f"Total, average, and std durations per file have been saved to {output_csv_path}")